# Reconnaissance faciale via $k$ plus proches voisins ($k$-NN)

>## Partie 2: Reconnaissance faciale en temps réel

## Librairies

In [1]:
import pickle
import numpy as np

import cv2

import tensorflow as tf

from sklearn.preprocessing import LabelEncoder

/Library/Python/3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Algorithme

In [2]:
with open('data/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open('data/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

Shape of visages matrix -->  (60, 50, 50, 3)


In [3]:
encoder = LabelEncoder()
noms_numeriques = encoder.fit_transform(noms)

N = len(noms)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(N, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(visages, noms_numeriques, epochs=1000)  


Epoch 1/1000
2/2 [==============================] - 0s 30ms/step - loss: 101.0623 - accuracy: 0.0000e+00
Epoch 2/1000
2/2 [==============================] - 0s 20ms/step - loss: 81.9865 - accuracy: 0.2500
Epoch 3/1000
2/2 [==============================] - 0s 18ms/step - loss: 28.4541 - accuracy: 0.3333
Epoch 4/1000
2/2 [==============================] - 0s 17ms/step - loss: 4.5250 - accuracy: 0.5167
Epoch 5/1000
2/2 [==============================] - 0s 17ms/step - loss: 11.7009 - accuracy: 0.3333
Epoch 6/1000
2/2 [==============================] - 0s 17ms/step - loss: 1.4573 - accuracy: 0.6667
Epoch 7/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.8646 - accuracy: 0.6833
Epoch 8/1000
2/2 [==============================] - 0s 17ms/step - loss: 1.0732 - accuracy: 0.6833
Epoch 9/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.7322 - accuracy: 0.8333
Epoch 10/1000
2/2 [==============================] - 0s 17ms/step - loss: 0.3981 - accuracy: 0.9333


## Exécution

In [4]:
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            # visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            visage_redimensionne = cv2.resize(visage, (50, 50)).reshape(1, 50, 50, 3)
            
            predictions_numeriques = model.predict(visage_redimensionne)

            texte = encoder.inverse_transform(predictions_numeriques.argmax(axis=1))
            
            cv2.putText(trame, str(texte[0]), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
        
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()

2024-03-01 16:16:08.070 Python[6979:7486744] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


1/1 [==============================] - 0s 47ms/step


2024-03-01 16:16:09.585 Python[6979:7486744] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


1/1 [==============================] - 0s 10ms/step


**Exemple d'exécution du système...**

<img src="moi.png" width="400" height="auto" />